In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from datetime import date, datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Logger import Logger
import WUBRG
import consts
import settings
from LineColors import LineColors
from FormatMetadata import SETS, FORMATS, SET_CONFIG
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata
from RawDataFetcher import RawDataFetcher
from RawDataHandler import RawDataHandler
from FramedData import FramedData

LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

In [ ]:
settings.ROLL

# Objects

In [ ]:
class FramedDataFuncs:   
    def __init__(self, SET, FORMAT, LOGGER=None):
        self._SET = SET
        self._FORMAT = FORMAT
        if LOGGER is None:
            LOGGER = Logger(Logger.FLG.DEFAULT)
        self.LOGGER = LOGGER
        self._DATA = FramedData(SET, FORMAT, self.LOGGER)
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll)       
        
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, roll)

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, roll)
    
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name)
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols, min_colors, max_colors)

    
    
    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date='2022-02-10')
        last = inner_func(date='2022-02-15')
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
        diff['Δ ATA'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [ ]:
class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    def get_games_played(self, deck_color):
        if deck_color: return self._DATA.deck_archetype_frame(deck_color=deck_color, summary=True)['Games'].sum()
        else: return self._DATA.deck_group_frame(name='All Decks', summary=True)['Games']
    
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self._DATA.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self._DATA.deck_group_frame(date=day, summary=True).loc[arch]['Win %']   
    
    def get_archetype_frame(self, colors, roll=None):
        if roll is None: roll = settings.ROLL
        win_rate_frame = self._DATA.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = settings.ROLL
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: 
            roll = settings.ROLL
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetype Winrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Win Percent")
        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = settings.ROLL
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: 
            roll = settings.ROLL
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetpye Playrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Percent of Metagame")

In [ ]:
class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA   

    def get_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = settings.ROLL
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[['GIH WR', 'ALSA', '# GP', 'IWD']]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = settings.ROLL
        rolling = self.get_card_summary(card_name, colors, roll)
        title = f"{self._DATA.SET} - {self._DATA.FORMAT}\n{card_name}"

        col_filt = f"Color Filter: {colors}"
        rol_filt = f"Rolling Average: {roll} Days"

        if colors and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif colors:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"

        rolling.plot(subplots=True, layout=(2,2), figsize=(12,8), title=title)
        plt.xlabel("Date")
      
    def get_pick_stats(self, card_name, roll=None):
        if roll is None: roll = settings.ROLL
        taken_data = self._DATA.card_frame(name=card_name, deck_color='')[['ALSA', 'ATA']]
        taken_data.index = [tup[0][5:] for tup in taken_data.index]
        taken_data = taken_data.rolling(window=roll, min_periods=1, center=True).mean()
        return taken_data
    
    def plot_pick_stats(self, card_name, roll=None):
        if roll is None: roll = settings.ROLL
        taken_data = self.get_pick_stats(card_name, roll)

        mx = min(max(taken_data.max()) + 0.25, 15) 
        mn = max(min(taken_data.min()) - 0.25, 1)
        tit_str = f"{self._DATA.SET} - {self._DATA.FORMAT}\n{card_name}"

        taken_data.plot(ylim=(mx, mn), grid=True, title=tit_str)
        plt.xlabel("Date")
        plt.ylabel("Pick Number")
    
    def card_archetype_performance(self, card_name):
        d = dict()
        d['AVG'] = self._DATA.card_frame(card_name, '', summary=True)
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self._DATA.card_frame(card_name, col, summary=True)

        test_frame = pd.DataFrame.from_dict(d).T
        return test_frame
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        ret = ret[WUBRG.COLOR_GROUPS]  #Re-orders the columns in WUBRG order
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

In [ ]:
class SetManager:
    def __init__(self, SET, LOGGER=None):
        if LOGGER is None:
            LOGGER = Logger(LOG_LEVEL)
        self.LOGGER = LOGGER
        self._SET = SET
        self._DATA = {f: FramedDataFuncs(SET, f, self.LOGGER) for f in FORMATS}
    
    
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self.BO1.check_for_updates()   
        self.BO3.check_for_updates()   
        self.QD.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self.BO1.reload_data()   
        self.BO3.reload_data()   
        self.QD.reload_data()  
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
            
    @property
    def BO1(self):
        """Premier Draft data."""
        return self._DATA['PremierDraft']
    
    @property
    def BO3(self):
        """Traditional Draft data."""
        return self._DATA['TradDraft']
    
    @property
    def QD(self):
        """Quick Draft data."""
        return self._DATA['QuickDraft']

# Initialization

In [ ]:
set_data = SetManager('NEO')

In [ ]:
set_data.check_for_updates()

## Current Tests

In [ ]:
import requests
from time import sleep
from datetime import date, time, datetime, timedelta
import pandas as pd

from Logger import Logger

class CallScryfall():
    _TRIES = 5
    _FAIL_DELAY = 60
    _SUCC_DELAY = 1
    
    _BASE_URL = 'https://api.scryfall.com/'
    
    def __init__(self, LOGGER=None):
        if LOGGER is None:
            LOGGER = Logger()
        self.LOGGER = LOGGER
        pass


    def _fetch(self, url: str) -> object:
        """
        Attempts to get json data from a url.
        :param url: The url to get data from
        :return: A json object or None
        """
        success = False
        count = 0

        while not success:
            count += 1

            try:
                self.LOGGER.log(f'Attempting to get data from {url}.', Logger.FLG.DEBUG)
                response = requests.get(url)
                data = response.json()

                success = True
                sleep(self._SUCC_DELAY)
                return data
            except:
                if count < self._TRIES:
                    self.LOGGER.log(f'Failed to get data. Trying again in {self._FAIL_DELAY} seconds.', Logger.FLG.DEFAULT)
                    sleep(self._FAIL_DELAY)
                    continue
                else:
                    self.LOGGER.log(f'Failed to get data after {self._TRIES} attempts.', Logger.FLG.ERROR)
                    self.LOGGER.log(f'Failed URL: {url}', Logger.FLG.ERROR)
                    return None        
    
    
    def get_set_cards(self, SET):
        cards = []
        next_page = True
        url = f'{self._BASE_URL}cards/search?format=json&include_extras=false&include_multilingual=false&order=set&page=1&q=e%3A{SET}&unique=cards'
        
        while next_page:
            response = self._fetch(url)
            cards += response['data']
            if response['has_more']:
                url = response['next_page']
            else:
                next_page = False
        
        return cards
    
    def get_card_by_name(self, NAME):
        """
        Gets card data from scryfall based on a name. Scryfall's fuzzy filter is
        used to handle imprecise queries and spelling errors.
        :param raw_card_name: The card name provided by a user
        :return: A card info struct which contains card data, and an error
        message if a problem occurred.
        """
        card_info = dict()
        card_info['name'] = NAME

        # Attempt to get information on the card.
        try:
            response = self._fetch(f'{self._BASE_URL}cards/named?fuzzy={NAME}')

            # If is not a card, do some processing and return the struct with some information.
            if response['object'] != 'card':
                # If the response type is an error, use that as the message.
                if response['object'] == 'error':
                    if response['details'][:20] == 'Too many cards match':
                        card_info['err_msg'] = f'Error: Multiple card matches for "{NAME}"'
                    else:
                        card_info['err_msg'] = f'Error: Cannot find card "{NAME}"'
                # If the search return a non-card, add that as the error message.
                else:
                    card_info['err_msg'] = f'Error: "{NAME}" returned non-card'
                return card_info
        # If an exception occurs, print it, and add an error massage to the struct.
        except Exception as ex:
            self.LOGGER.log(ex, Logger.FLG.ERROR)
            card_info['err_msg'] = f'Error: Failed to query Scryfall for {NAME}\r\n{ex}'
            return card_info
            
        # If no problems occurred, get the relevant card info and populate the card_info_struct
        to_remove = to_remove = ['oracle_id', 'mtgo_id', 'multiverse_ids', 'tcgplayer_id', 'cardmarket_id', 
                                 'lang', 'released_at', 'highres_image', 'image_status', 
                                 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 
                                 'oversized', 'promo', 'reprint', 'variation', 
                                 'set_id', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 
                                 'rulings_uri', 'prints_search_uri', 'related_uris', 'prices', 'purchase_uris',
                                 'digital', 'card_back_id', 'border_color', 'frame', 'full_art', 'textless',
                                 'booster', 'story_spotlight', 'edhrec_rank', 'flavor_text', 'artist', 'artist_ids', 'artist_id',
                                 'illustration_id', 'preview', 'security_stamp', 'image_uris', 'uri', 'scryfall_uri']
        for prop in to_remove:
            if prop in response:
                del response[prop]
        if 'card_faces' in response:
            for face in response['card_faces']:
                for prop in to_remove:
                    if prop in face:
                        del face[prop]
        return response

In [ ]:
from enum import unique

#@unique
class CardLayouts():
    NORMAL = 0
    ADVENTURE = 1
    SLPIT = 2
    TRANSFORM = 3
    MODAL_DFC = 4


class Card():
    # TODO: Consider expanding this to conatin more information.
    # Pieces of information could be Keywords, Types, Faces, etc.
    
    API_URL = 'https://api.scryfall.com/cards/'
    SCRY_URL = 'https://scryfall.com/card/'
    IMG_URL = 'https://c1.scryfall.com/file/scryfall-cards/'
    
    def from_name(name):
        scry = CallScryfall()
        json = scry.get_card_by_name(name)
        return Card(json)
    
    def _handle_layout(self, layout):
        # This property is likely very important in future-proofing this object, 
        # when handling cards which have any deviation from the standard frame.
        # https://scryfall.com/docs/api/layouts
        # relevant_layouts = ['normal', adventure', 'split', 'transform', 'modal_dfc']
        if layout == 'normal': 
            self.LAYOUT = CardLayouts.NORMAL
            self.TWO_SIDED = False
            self.SPLIT = False
        elif layout == 'adventure': 
            self.LAYOUT = CardLayouts.ADVENTURE
            self.TWO_SIDED = False
            self.SPLIT = True
        elif layout == 'split': 
            self.LAYOUT = CardLayouts.SLPIT
            self.TWO_SIDED = False
            self.SPLIT = True
        elif layout == 'transform': 
            self.LAYOUT = CardLayouts.TRANSFORM
            self.TWO_SIDED = True
            self.SPLIT = False
        elif layout == 'modal_dfc': 
            self.LAYOUT = CardLayouts.MODAL_DFC
            self.TWO_SIDED = True
            self.SPLIT = False
        else: raise Exception(f"Unknown layout '{layout}'")

    def __init__(self, json):
        if json['object'] != 'card':
            raise Exception("Invalid JSON provided! Object type is not 'card'")
        
        # Card ID info
        self.ID = json['id']
        self.ARENA_ID = json['arena_id']
        self.SET = json['set'].upper()
        self.RARITY = json['rarity']
        self.FULL_NAME = json['name']
        self.NUMBER = json['collector_number']
        self._handle_layout(json['layout'])

        # TODO: Determine the properties required for a card face, and how to mave the main card
        # access the appropriate ones based on type.
        
        # Card Info
        self.CMC = json['cmc']
        #type_line
        self.COLOR_IDENTITY = "".join(json['color_identity'])
        #keywords
        
        if 'card_faces' in json: 
            self.NAME = json['card_faces'][0]['name']
            self.MANA_COST = json['card_faces'][0]['mana_cost']
        else: 
            self.NAME = self.FULL_NAME
            self.MANA_COST = json['mana_cost']
            
            
        

        
    @property
    def API(self):
        """Link for the API call of the card"""
        return f"{self.API_URL}{self.ID}"

    @property
    def URI(self):
        """Shortened link to the Scryfall page for the card"""
        return f"{self.SCRY_URL}{self.SET.lower()}/{self.NUMBER}"
    
    sizes = ['small', 'normal', 'large', 'png', 'art_crop', 'border_crop']
    def IMAGE_URL(self, size='normal', face='front'):
        """Returns a link to the """
        return f"{self.IMG_URL}{size}/{face}/{self.ID[0]}/{self.ID[1]}/{self.ID}.jpg"

    def __str__(self):
        return self.FULL_NAME

    def __repr__(self):
        return self.FULL_NAME


In [ ]:
scry = CallScryfall()

In [ ]:
normal = scry.get_card_by_name('Jukai Preserver')
normal

In [ ]:
transform = scry.get_card_by_name('Boseiju reaches Skyward')
transform

In [ ]:
adventure = scry.get_card_by_name('Bonecrusher Giant')
adventure

In [ ]:
modal = scry.get_card_by_name('Shatterskull Smashing')
modal

In [ ]:
split = scry.get_card_by_name('Connive')
split

In [ ]:
card = Card(card)
card.FULL_NAME

In [ ]:
print(card.IMAGE_URL('large'))

In [ ]:
cards = scry.get_set_cards('NEO')
cards

In [ ]:
from functools import cmp_to_key
    
def compress_date_range_data(start_date, end_date, card_name=None):
    # Set up dictionaries for quicker sorting.
    COLOR_INDEXES = { WUBRG.COLOR_GROUPS[x]: x for x in range(0, len(WUBRG.COLOR_GROUPS)) }
    #TODO: Have the cards be pulled from Scryfall via SetMetadata
    CARDS = list(set_data.BO1.card_frame(summary=True, deck_color='').reset_index(level=0).index)    
    CARD_INDEXES = {CARDS[x]: x for x in range(0, len(CARDS))}

    # Creating a custom sorting algortihm
    def compare(pair1, pair2):
        # Convert the colors and names into numeric indexes
        color1, name1 = pair1
        col_idx1 = COLOR_INDEXES[color1]
        name_idx1 = CARD_INDEXES[name1]
        color2, name2 = pair2
        col_idx2 = COLOR_INDEXES[color2]
        name_idx2 = CARD_INDEXES[name2]

        # Sort by deck colour than card number.
        if col_idx1 == col_idx2:
            if name_idx1 < name_idx2: return -1
            else: return 1
        if col_idx1 < col_idx2: return -1
        else: return 1

    compare_key = cmp_to_key(compare)

    # The columns which have win percents.
    percent_cols = ['GP', 'OH', 'GD', 'GIH', 'GND']
    
    # Get the relevant dates (and card)
    frame = set_data.BO1.card_frame(card_name, date=slice(start_date, end_date)).copy()
    
    # Calculate helper stats to recalculate value later.
    frame['ALSA SUM'] = frame['ALSA'] * frame['# Seen']
    frame['ATA SUM'] = frame['ATA'] * frame['# Picked']
    for col in percent_cols:
        frame[f'# {col} WINS'] = frame[f'# {col}'] * frame[f'{col} WR']
    
    # Take the expanded frame, and drop the dates.
    frame = frame.reset_index(level=0)
    frame = frame.drop('Date', axis=1)

    # Sum the frame by deck colours and cards.
    temp = frame.groupby(['Deck Colors', 'Name']).max() #Used to preserve color and rarity.
    frame = frame.groupby(['Deck Colors', 'Name']).sum()
    frame['Color'] = temp['Color']
    frame['Rarity'] = temp['Rarity']

    # Re-caulculate the stats based on the processing from above.
    frame['ALSA'] = frame['ALSA SUM'] / frame['# Seen']
    frame['ATA'] = frame['ATA SUM'] / frame['# Picked']
    for col in ['GP', 'OH', 'GD', 'GIH', 'GND']:
        frame[f'{col} WR'] = frame[f'# {col} WINS'] / frame[f'# {col}']
    frame['IWD'] = frame['GIH WR'] - frame['GND WR']
    
    # Trim the helper columns from the epanded frame.
    summed = frame[['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']]
    idx = list(summed.index)
    idx.sort(key=compare_key)
    summed = summed.set_index([idx])
    
    return summed

In [ ]:
comp_new = compress_date_range_data('2022-02-28', '2022-03-10')
comp_new

In [ ]:
set_data.BO1.compress_date_range_data('2022-02-28', '2022-03-10')

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_qudrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_qudrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_qudrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_qudrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
set_data.BO1.card_archetype_performance("Era of Enlightenment")

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Generate a summary frame from X last days (or from between any two days)
 - Have cardnames pull from a list from metadata.
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Expend on log handling in existing objects.
- Generate a list of cards for a set using Scryfall.
 - Better organize card data using objects and helper functions.

# Data Graphing and Display

## Card Summary

In [ ]:
set_data.BO1.plot_card_summary("Fade into Antiquity", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("Befriending the Moths", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("Michiko's Reign of Truth", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("The Fall of Lord Konda", roll=3)

In [ ]:
set_data.BO1.plot_card_summary('Sunblade Samurai', roll=3)

In [ ]:
set_data.BO1.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='RM', play_lim=0.005)

## Card Pick Order Stats

In [ ]:
set_data.BO1.plot_pick_stats('Imperial Oath', 3)

In [ ]:
set_data.BO1.plot_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor']
for card in to_graph:
    set_data.BO1.plot_pick_stats(card, 1)

## Archetype Winrate History

In [ ]:
set_data.BO1.get_archetype_winrate_history()

In [ ]:
set_data.BO1.plot_archetype_winrate_history('')

In [ ]:
set_data.BO3.plot_archetype_winrate_history('')

## Archetype Playrate History

In [ ]:
set_data.BO1.get_archetype_playrate_history()

In [ ]:
set_data.BO1.plot_archetype_playrate_history('')

In [ ]:
set_data.BO3.plot_archetype_playrate_history('')

## Card Pick Order Changes

In [ ]:
diff = set_data.BO1.compare_card_evaluations('2022-02-17', '2022-03-03')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'Δ ALSA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)